In [36]:
%serialconnect


Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [37]:
# developments experimenting with using the android as a hotspot
import network
# apparently the hotspotting android device will always be 192.168.43.1

In [38]:
# Create access point for the phone to log onto
si = network.WLAN(network.STA_IF) 
si.active()

In [39]:
wconn, wpass = b'BV6000', b'beckaaaa'
bwconnexists = bool(l  for l in si.scan()  if l[0] == wconn)

scandone


In [40]:
import time
if bwconnexists:
    si.connect(wconn, wpass)
    while not si.isconnected():
        time.sleep_ms(100)

scandone


In [41]:
print(si.ifconfig())

('192.168.43.11', '255.255.255.0', '192.168.43.1', '192.168.43.1')


In [42]:
si2 = network.WLAN(network.AP_IF) 
si2.active(True)      
print(si2.ifconfig())


('192.168.4.1', '255.255.255.0', '192.168.4.1', '192.168.43.1')


In [49]:
print(si2.active())


True


In [2]:
import time, machine
pblue = machine.Pin(2, machine.Pin.OUT)
pboot = machine.Pin(0, machine.Pin.IN)

for i in range(21):
    pblue.value(pboot.value())
    time.sleep(0.6)


.

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 7, in <module>
KeyboardInterrupt: 


In [9]:
q = machine.Pin(0, machine.Pin.IN)
print(q.value())


1


In [10]:
# This is the current main as hotspot
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [11]:
%sendtofile main.py

import network, socket
import time, machine

pled = machine.Pin(2, machine.Pin.OUT)
pboot = machine.Pin(0, machine.Pin.IN)

port = 9019
sockudp = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sockudp.settimeout(0.1)

wconn, wpass = b'BV6000', b'beckaaaa'
si = network.WLAN(network.STA_IF) 
si.active(True)
bwconnexists = bool(l  for l in si.scan()  if l[0] == wconn)
if bwconnexists:
    si.connect(wconn, wpass)
    while not si.isconnected():
        pled.value(1-pled.value())
        time.sleep_ms(100)
    udpaddr = ("192.168.43.1", port)
    
else:  # Create access point for the phone to log onto
    si = network.WLAN(network.AP_IF) 
    si.active(True)      
    #espname = "ESP_%X%X%X" % tuple(si.config("mac")[-3:])  # esp32 version
    espname = "MicroPython_%x%x%x" % tuple(si.config("mac")[-3:])
    print("\nConnect to wifi {}".format(espname))
    udpaddr = None
    sockudp.bind(("192.168.4.1", port))

print(si.ifconfig())
tudptimeout = 0

def dwrite(mess):
    global udpaddr
    if udpaddr is not None:
        try:
            sockudp.sendto(mess, udpaddr)
        except OSError as e:
            print("dwrite", e)
            if not bwconnexists:
                udpaddr = None

def dflush():
    global udpaddr
    if udpaddr is None:
        try:
            rmess, udpaddr = sockudp.recvfrom(100)
            print(rmess, udpaddr)
            return udpaddr
        except OSError as e:
            print("dflush", e)
    return None


Sent 53 lines (1464 bytes) to main.py.


In [12]:
%sendtofile --append main.py

import urandom

ubs = bytearray("Ut00000000i00\n")
mubs = memoryview(ubs)
prevflushstamp = 0
nextledonstamp = 0

while True:
    tstamp = time.ticks_ms()
    
    if tstamp > nextledonstamp:
        pv = (0 if pled.value() else 1)
        pled.value(pv)
        mubs[2:10] = b"%08X" % tstamp
        mubs[12] = 48+pv #'0'or'1'
        mubs[11] = 48+(1-pboot.value())
        dwrite(ubs)
        nextledonstamp = tstamp + (500+urandom.getrandbits(9))//(10 if pv else 1)
            
    if tstamp > prevflushstamp + 2000:
        prevflushstamp = tstamp
        k = dflush()
        if isinstance(k, tuple):
            for i in range(40):
                pled.value(i%2)
                time.sleep_ms(50)
            print(["UDP", k[0], k[1]])  
            

Sent 28 lines (758 bytes) to main.py.


#### Use this if receiving from UDP socket and saving

import socket
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.settimeout(3)
ipnum = "192.168.4.3"
port = 9019
#s.bind((ipnum, port))   
s.sendto(b"HiThere", ("192.168.4.1", port))
print(s.recvfrom(200))


In [38]:
print(urandom.getrandbits(9))


291
